In [ ]:
using Revise
using Pkg
Pkg.activate(".")
using Plots
using BalanceOfPlantSurogate

In [ ]:
# get the model and data
bop_sur_rankine = BalanceOfPlantSurogate.BOPSurogate(:rankine;data="BalanceOfPlantHypercubeN=10000.csv")
bop_sur_brayton = BalanceOfPlantSurogate.BOPSurogate(:brayton;data="BalanceOfPlantHypercubeN=10000.csv")

BalanceOfPlantSurogate.predict_thermal_efficiency_fractions(bop_sur_rankine,1e8,0.5,0.2), BalanceOfPlantSurogate.predict_thermal_efficiency_fractions(bop_sur_brayton,1e8,0.5,0.2)

In [ ]:
# plot the data
scatter(log10.(bop_sur_brayton.data.total_heat_load),bop_sur_brayton.data.thermal_efficiency_plant,label="brayton")
scatter!(log10.(bop_sur_rankine.data.total_heat_load),bop_sur_rankine.data.thermal_efficiency_plant,ylim=(-1.0,1.0),label="rankine",ylabel="thermal plant efficiency",xlabel="total heat supplied")


In [ ]:
# Display data with sliders

function log10range(start_value, stop_value, num_points)
    return 10 .^ LinRange(log10(start_value), log10(stop_value), num_points)
end

total_heat_supplied = log10range(1e7,5e9,50)

@manipulate for breeder_fraction in 0.5:0.025:0.9, divertor_fraction in 0.1:0.25:1.0
    efficiencies = [BalanceOfPlantSurogate.predict_thermal_efficiency_fractions(bop_sur_brayton, p1, breeder_fraction, divertor_fraction) for p1 in total_heat_supplied]
    plot(total_heat_supplied,efficiencies,marker=:circle, markersize=1, label="brayton")

    efficiencies = [BalanceOfPlantSurogate.predict_thermal_efficiency_fractions(bop_sur_rankine, p1, breeder_fraction, divertor_fraction) for p1 in total_heat_supplied]
    plot!(total_heat_supplied,efficiencies,marker=:circle, markersize=1, label="rankine")
    
    plot!(xlabel="total heat supplied [W]",ylabel="thermal plant efficiency", ylim=(0.0,0.9), xscale=:log10,)
end


In [ ]:
#divertor_heat_fraction = 0.2  # You can adjust this value
cycle_lim = (0.0,0.6)
#total_heat_supplied = log10range(1e5,5e9,20)

#@manipulate for breeder_fraction in 0.5:0.05:0.9, divertor_fraction in 0.1:0.05:1.0
@manipulate for total_heat in log10range(1e5,5e9,20)
    # Create a grid of values
    breed = collect(0.5:0.05:0.9)
    div = collect(0.1:0.05:1.0)
    efficiencies = [BalanceOfPlantSurogate.predict_thermal_efficiency_fractions(bop_sur_brayton, total_heat, p1,p2)
                    for p1 in breed, p2 in div]
#    @show efficiencies
    surface(breed, div, efficiencies',
            xlabel="total heat supplied [W]", ylabel="breeder heat fraction", zlabel="Thermal Efficiency",xscale=:log10,
            title="Efficiency vs. total heat supplied and breeder heat fraction RANKINE cycle",zlim=cycle_lim)
end